# ДЗ

Задание 1 [3 балла]. Создайте схему БД "Университет".

Она должна включать в себя минимально следующие отношения:

    - студенты (минимальные атрибуты: ФИО, номер зачётки, дата рождения, группа, год поступления)
    - посещаемые курсы (минимальные атрибуты: название курса, тип курса, дата введения в программу)
    - преподаватели (минимальные атрибуты: ФИО, дата рождения, дата начала работы в нашем университете, дата начала трудовой деятельности)
Атрибуты отношений должны быть осмысленными и соответствовать тематике.

Задание 2 [1 балл]. Заполните вашу схему фейковыми записями - в отношении Студенты должно быть не менее 150 строк

В остальных - не менее 20 строк.

In [1]:
import random
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import sqlite3

In [2]:
path = './data/'
filestud='students.csv'
fileteach='teachers.csv'
filesubj='subjects.csv'
delim=','
obligatory_num = 6
optional_min = 3
optional_max = 7
year1 = 2017
year2 = 2022
group_prefix = ['ПМИ','ПМ','ИБ','ИВТ','ПИ']
# Ограничиваем диапазон дат рождения студентов
start_dt = dt.strptime('01.01.1998', '%d.%m.%Y')
end_dt = dt.strptime('31.12.2004', '%d.%m.%Y')
# Ограничиваем диапазон дат рождения преподавателей
start_dt2 = dt.strptime('01.01.1945', '%d.%m.%Y')
end_dt2 = dt.strptime('31.12.1990', '%d.%m.%Y')
# Ограничиваем диапазон дат начала работы в ВУЗе
start_dt3 = dt.strptime('01.01.2000', '%d.%m.%Y')
end_dt3 = dt.strptime('31.08.2020', '%d.%m.%Y')
# Ограничиваем диапазон дат начала рабочего стажа
start_dt4 = dt.strptime('01.01.1970', '%d.%m.%Y')
end_dt4 = dt.strptime('31.08.2020', '%d.%m.%Y')

def get_random_year():
    return random.randrange(year1,year2+1)

# Функция, добавляющая годы к дате, чтобы прибавлять 16 лет к дате рождения для генерации даты начала труд.деятельности
def add_years(d, years):
    try:
        return d.replace(year = d.year + years)
    except ValueError:
        return d + (date(d.year + years, 1, 1) - date(d.year, 1, 1))

# Генерируем случайные даты
def get_random_date(start, end):
    delta = end - start
    return start + timedelta(random.randint(0, delta.days))
 
# Генерируем название группы формата % код специальности % - % цифра от 1 до 4 % - % год поступления / 100 %
def get_random_group(year):
    i = random.randrange(len(group_prefix))
    j = random.randrange(1,4)
    return group_prefix[i] + '-' + str(j) + '-' + str(year % 100)

random.seed(2)
# Студенты
# Фамилия, Имя, Отчество, Номер зачётки ,Дата рождения, Группа, Год поступления
dstudents = pd.read_csv(path + filestud, sep=delim, decimal=',',  encoding='utf-8')
for index, row in dstudents.iterrows():
    tmp = get_random_year()
    dstudents.loc[index,'year'] = str(tmp)
    dstudents.loc[index,'rb_num'] = str(tmp * 1000 + index + 1)
    dstudents.loc[index,'birthday'] = str(get_random_date(start_dt, end_dt))[0:10]
    dstudents.loc[index,'group_nm'] = get_random_group(tmp)
dstudents.insert(0,column='id',value=dstudents.index+1001)

# Преподаватели
# Фамилия, Имя, Отчество, Дата рождения, Дата начала работы в ВУЗе, Дата начала трудовой деятельности
dteachers = pd.read_csv(path + fileteach, sep=delim, decimal=',',  encoding='utf-8')
for index, row in dteachers.iterrows():
    tmp = get_random_year()-3
    bday = get_random_date(start_dt2, end_dt2) # Дата рождения
    wday = get_random_date(add_years(bday, 16), end_dt4) # Начало работы
    dteachers.loc[index,'birthday'] = str(bday)[0:10]
    dteachers.loc[index,'work_exp'] = str(wday)[0:10]
    # Дата начала работы в ВУЗе не может быть раньше даты начала трудовой деятельности
    dteachers.loc[index,'work_start_dt'] = str(get_random_date(wday, end_dt3))[0:10]
dteachers.insert(0,column='id',value=dteachers.index+301)

# Посещаемые курсы
# Название, Тип, Дата введения
dsubjects = pd.read_csv(path + filesubj, sep=delim, decimal=',',  encoding='utf-8')
dsubjects.insert(0,column='id',value=dsubjects.index+1)
for index, row in dsubjects.iterrows():
    tmp = dsubjects.loc[index,'start_dt']
    dsubjects.loc[index,'start_dt'] = tmp[6:10]+"-"+tmp[3:5]+"-"+tmp[0:2]
    
teacher_x_subject = pd.DataFrame(columns=['teacher_id','subject_id'])
display(dstudents)
display(dteachers)
display(dsubjects)

# Формируем таблицу отношений Преподаватель-Посещаемые курсы
for index, row in dteachers.iterrows():
    n = random.randrange(0,3)
    if n == 0: # Математические науки
        q = random.randrange(1,4)
        r = random.sample(range(1, 6), q)
        for k in r:
            teacher_x_subject.loc[len(teacher_x_subject.index)] = [dteachers.loc[index,'id'], k]
    elif n == 1: # Информатика
        q = random.randrange(1,5)
        r = random.sample(range(6, 17), q)
        for k in r:
            teacher_x_subject.loc[len(teacher_x_subject.index)] = [dteachers.loc[index,'id'], k]
    else: # Единственный предмет
        k = random.randrange(17,21)
        teacher_x_subject.loc[len(teacher_x_subject.index)] = [dteachers.loc[index,'id'], k]

# Формируем таблицу отношений Студент-Посещаемые курсы
student_x_subject = pd.DataFrame(columns=['student_id','subject_id'])
obligatory_list = list(dsubjects.loc[dsubjects['type'] == 'Обязательный']['id'])
optional_list = list(dsubjects.loc[dsubjects['type'] == 'По выбору']['id'])
for index, row in dstudents.iterrows():
    for k in random.sample(obligatory_list, obligatory_num): # Обязательные предметы
        student_x_subject.loc[len(student_x_subject.index)] = [dstudents.loc[index,'id'], k]
    for k in random.sample(optional_list, random.randrange(optional_min, optional_max+1)): # Предметы по выбору
        student_x_subject.loc[len(student_x_subject.index)] = [dstudents.loc[index,'id'], k]
        
display(teacher_x_subject)
display(student_x_subject)

,id,surname,name,middlename,rb_num,birthday,group_nm,year
0,1001,Агапова,Ника,Матвеевна,2017001,1999-01-11,ПМИ-2-17,2017
1,1002,Агафонов,Руслан,Андреевич,2018002,2001-06-16,ИБ-3-18,2018
2,1003,Агеев,Матвей,Матвеевич,2018003,2004-10-21,ПМИ-3-18,2018
3,1004,Аксенов,Григорий,Дмитриевич,2022004,1999-10-11,ИВТ-3-22,2022
4,1005,Александров,Вячеслав,Даниилович,2020005,2003-09-17,ИБ-3-20,2020
...,...,...,...,...,...,...,...,...
295,1296,Шарова,Виктория,Романовна,2018296,1999-08-25,ПМ-3-18,2018
296,1297,Шевцов,Константин,Сергеевич,2022297,1999-12-14,ИВТ-2-22,2022
297,1298,Шувалов,Роман,Львович,2020298,1999-12-09,ИВТ-2-20,2020
298,1299,Яковлева,Антонина,Тимофеевна,2019299,2004-05-25,ИВТ-2-19,2019


,id,surname,name,middlename,birthday,work_start_dt,work_exp
0,301,Авдеева,Елизавета,Николаевна,1980-12-17,2003-05-22,1998-10-07
1,302,Агафонов,Фёдор,Максимович,1954-02-03,1980-10-28,1980-05-06
2,303,Акимова,Анна,Марковна,1973-12-08,2018-11-06,2017-01-27
3,304,Аксенов,Тимур,Никитич,1981-06-05,2012-09-27,2005-06-09
4,305,Андреев,Михаил,Вадимович,1988-01-31,2020-01-21,2013-02-07
...,...,...,...,...,...,...,...
75,376,Ткачев,Никита,Робертович,1968-11-20,2006-11-08,1996-05-30
76,377,Филиппов,Андрей,Денисович,1972-10-04,2015-03-31,2002-08-14
77,378,Фролов,Денис,Арсеньевич,1989-09-20,2018-09-24,2016-07-25
78,379,Чернов,Александр,Александрович,1980-04-22,2018-09-18,2004-01-28


,id,title,type,start_dt
0,1,Математический анализ,Обязательный,2000-08-31
1,2,Линейная аглебра,Обязательный,2000-08-31
2,3,Дискретная математика,Обязательный,2000-08-31
3,4,Теория верояятностей и математическая статистика,Обязательный,2000-08-31
4,5,Функциональный анализ,Обязательный,2003-08-31
5,6,Теоретическая информатика,Обязательный,2001-08-31
6,7,Объектно-ориентированное программирование,По выбору,2010-08-31
7,8,Теория управления,По выбору,2013-08-31
8,9,Введение в базы данных,По выбору,2009-08-31
9,10,Алгоритмизация и программмирование,По выбору,2013-08-31


,teacher_id,subject_id
0,301,20
1,302,19
2,303,17
3,304,17
4,305,15
...,...,...
117,377,3
118,378,5
119,378,2
120,379,1


,student_id,subject_id
0,1001,17
1,1001,2
2,1001,4
3,1001,5
4,1001,6
...,...,...
3292,1300,10
3293,1300,15
3294,1300,7
3295,1300,14


In [3]:
conn = sqlite3.connect('university') 
dstudents.to_sql('students',conn,None,if_exists='replace')
dteachers.to_sql('teachers',conn,None,if_exists='replace')
dsubjects.to_sql('subjects',conn,None,if_exists='replace')
teacher_x_subject.to_sql('teacher_x_subject',conn,None,if_exists='replace')
student_x_subject.to_sql('student_x_subject',conn,None,if_exists='replace')

3297

Задание 3 [1 балл]. Выведите топ-10 курсов, которые ведут наименьшее число преподавателей. Расположите в порядке возрастания числа преподавателей.

In [4]:
sql_query = pd.read_sql_query('''SELECT sb.title AS title,
                                        COUNT(txs.teacher_id) AS teachers_num
                                        FROM subjects AS sb 
                                        JOIN teacher_x_subject AS txs ON sb.ID = txs.SUBJECT_ID
                                        GROUP BY 1
                                        ORDER BY 2
                                        LIMIT 10''',conn)
df = pd.DataFrame(sql_query, columns = ['title', 'teachers_num'])
display(df)

,title,teachers_num
0,Распределённые системы,1
1,Введение в базы данных,2
2,БЖД,3
3,Системы и сети передачи информации,3
4,Математическая инженерия в науке и бизнесе,4
5,Основы радиоэлектроники и связи,4
6,Теоретическая механика,4
7,Объектно-ориентированное программирование,5
8,Промышленная разработка,5
9,Теория управления,5


Задание 4 [2 балла]. Напишите запрос, который выведет топ-10 преподавателей, которые преподают у наибольшего числа студентов. Расположите их в порядка убывания числа студентов.

In [5]:
sql_query = pd.read_sql_query('''SELECT t.surname||' '||t.name||' '||t.middlename AS teacher_fio,
                                        COUNT(DISTINCT sxs.student_id) AS stud_cov
                                        FROM teachers AS t
                                        JOIN teacher_x_subject AS txs ON t.id = txs.teacher_id
                                        JOIN student_x_subject AS sxs ON txs.subject_id = sxs.subject_id
                                        GROUP BY 1
                                        ORDER BY 2 DESC, 1
                                        LIMIT 10''',conn)
df = pd.DataFrame(sql_query, columns = ['teacher_fio', 'stud_cov'])
display(df)

,teacher_fio,stud_cov
0,Белоусов Кирилл Сергеевич,299
1,Клюева Валерия Кирилловна,299
2,Высоцкий Савелий Дмитриевич,298
3,Горбунов Артемий Александрович,298
4,Быков Захар Макарович,297
5,Овчинников Владимир Егорович,297
6,Плотникова София Ивановна,297
7,Андреев Михаил Вадимович,287
8,Раков Дмитрий Матвеевич,285
9,Елисеев Максим Лукич,282


Задание 5 [1.5 балла]. Для каждой группы студентов, поступивших не ранее 2019 года, выведите размах между максимальной и минимальной датами рождений студентов в этой группе.

In [6]:
sql_query = pd.read_sql_query('''SELECT group_nm,
                                        CAST(JulianDay(max_bd) - JulianDay(min_bd) AS integer) AS bd_diff_days
                                        FROM (SELECT group_nm,
                                                     MAX(birthday) AS max_bd,
                                                     MIN(birthday) AS min_bd
                                                     FROM students
                                                     WHERE year >= '2019' 
                                                     GROUP BY 1)
                                        WHERE bd_diff_days > 0 /* исключаем группы где менее 2 человек */
                                        GROUP BY 1''',conn)
df = pd.DataFrame(sql_query, columns = ['group_nm', 'bd_diff_days'])
display(df)

,group_nm,bd_diff_days
0,ИБ-1-20,1755
1,ИБ-1-21,1479
2,ИБ-1-22,1401
3,ИБ-2-20,2090
4,ИБ-2-22,319
5,ИБ-3-19,1719
6,ИБ-3-20,1176
7,ИБ-3-21,978
8,ИБ-3-22,788
9,ИВТ-1-19,1490


Задание 6 [1.5 балла]. Среди преподавателей, родившихся в период между самой ранней датой начала работы в нашем университете и самой ранней датой начала трудовой деятельности по всем преподавателям, определите самое популярное имя.

In [7]:
sql_query = pd.read_sql_query('''SELECT name,
                                        MAX(name_cnt) AS max_cnt
                                        FROM (SELECT name,
                                                     COUNT(id) AS name_cnt
                                                     FROM teachers
                                                     WHERE birthday BETWEEN (SELECT MIN(work_exp) FROM teachers)
                                                     AND (SELECT MIN(work_start_dt) FROM teachers)
                                                     GROUP BY 1) ''',conn)
                    
df = pd.DataFrame(sql_query, columns = ['name', 'max_cnt'])
display(df)

,name,max_cnt
0,Даниил,2


In [8]:
conn.close()